In [ ]:
# !pip install selenium
# !pip install BeautifulSoup4
# !pip install requests
# !pip install pandas
# !pip install webdriver_manager
# !pip install pytest-shutil

------

## Coupang Seller DB ETL

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import *

import subprocess
import shutil

from bs4 import BeautifulSoup as bs
from urllib.parse import urlparse, parse_qs

import pandas as pd
import time
import datetime
import warnings
import requests
import re

In [ ]:
import requests, openpyxl
from bs4 import BeautifulSoup
import re

excel_file = openpyxl.Workbook()
excel_sheet = excel_file.active
excel_sheet.append(['랭킹', '상품명', '판매가격', '상품상세링크'])

url = "https://www.coupang.com/np/search?component=&q=%EC%8B%A0%EC%9D%BC+%EC%84%9C%ED%81%98%EB%A0%88%EC%9D%B4%ED%84%B0&channel=user"
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.66 Safari/537.36"}
res = requests.get(url, headers = headers, verify=False)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

In [2]:
def find_css(css_selector):
    return browser.find_element(By.CSS_SELECTOR, css_selector)

def finds_css(css_selector):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath):
    return browser.find_element(By.XPATH, xpath)

def finds_xpath(xpath):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id):
    return browser.find_element(By.ID, e_id)

def find_tag(e_tag):
    return browser.find_element(By.TAG_NAME, e_tag)

def find_className(cn):
    return browser.find_element(By.CLASS_NAME, cn)

def finds_className(cn):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt):
    return browser.find_element(By.LINK_TEXT, lt)

In [3]:
def getNext(x, lst):
    next_el = None
    for i in range(0, len(lst)):
        if lst[i] == x:
            next_el = lst[i+1]
    return next_el

In [4]:
def getBefore(x, lst):
    before_el = None
    for i in range(0, len(lst)):
        if lst[i] == x:
            before_el = lst[i-1]
    return before_el

In [5]:
def delCookie():
    try:
        shutil.rmtree(r"C:\Users\mirae\jupyter notebook\shopping_crawling\Cookie")  #쿠키 / 캐쉬파일 삭제
    except FileNotFoundError:
        pass

In [74]:
def open_browser():
    subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 -incognito --user-data-dir="C:\Users\mirae\jupyter notebook\shopping_crawling\Cookie"')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--window-size=1080,800")
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path="chromedriver.exe")

    browser = webdriver.Chrome(service = chrome_service, options=options)

    browser.implicitly_wait(3)
    
    return browser

In [6]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--window-size=1080,800")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
    options.add_argument('incognito')
    # options.add_argument('headless')
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path="chromedriver.exe")
    browser = webdriver.Chrome(service=chrome_service, options=options)
    
    return browser

In [7]:
keywords = ['같이',
'해변',
'노란색',
'묘지',
'영재',
'헌신적',
'지혜',
'유상',
'필터',
'젊음',
'종지부',
'식견',
'가슴',
'오산',
'황소',]

In [8]:
error = []
seller_page_hrefs = []

In [9]:
final_info = [[''], [''],['']]

In [10]:
travel = []

In [ ]:
try:
    for keyword in keywords:
        try:
            browser = open_browser()
            browser.get(f'https://www.coupang.com/np/search?q={keyword}')
            soup = bs(browser.page_source, "html.parser")
            soup = soup.find_all(class_='search-product')
            try:
                for i in soup:
                    seller_page_hrefs.append('https://www.coupang.com'+(i.find("a").attrs['href']))
            except KeyError:
                print(i)
            except:
                print(i)
        except:
            pass

        try:
            for p in range(2, 28):
                url = f'https://www.coupang.com/np/search?q={keyword}&page={p}'
                try:
                    browser.get(url)
                except: continue

                soup = bs(browser.page_source, "html.parser")
                soup = soup.find_all(class_='search-product')

                try:
                    for i in soup:
                        seller_page_hrefs.append('https://www.coupang.com'+(i.find("a").attrs['href']))
                except KeyError:
                    print(i)
                except:
                    print(i)
        except:
            pass

        seller_page_hrefs = list(set(seller_page_hrefs))
        l = len(seller_page_hrefs)
        c = 0
        for i in seller_page_hrefs:
            print(l, c, keyword, i)
            c += 1
            if 'https://travel.coupang.com/' in i in i: 
                travel.append(i)
                print('pass')
                continue
            try:
                browser.get(i)
            except:
                continue

            soup_ad = bs(browser.page_source, "html.parser")
            if soup_ad.find('title').text == 'Access Denied':
    #             t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    #             browser.save_screenshot(f'./{t}.png')
                browser.close()
                time.sleep(1)
                browser = open_browser()
                print(soup_ad.find('title').text)
                i = getBefore(i, seller_page_hrefs)
                continue
            try:
                soup = bs(browser.page_source, "html.parser")
            except UnexpectedAlertPresentException:
                browser.switchTo().alert().accept();
                i = getBefore(i, seller_page_hrefs)

            soup = soup.find_all(class_='td-delivery-badge')
            try:
                soup[0].find('img').attrs['class']
                continue
            except AttributeError:
                pass
            except IndexError:
                continue

            time.sleep(1)
            element = find_tag('body')

            element.send_keys(Keys.END)
            time.sleep(1)

            tb = finds_css('table.prod-delivery-return-policy-table > tbody')
            if tb == []: continue
            ta =tb[-1].text.split('\n')

            name, cont, loca = None, None, None

            try:
                for i in ta:
    #                     print(i)
                    if '대표자' in i:
                        name = getNext(i, ta).split('/ ')[-1]
                    if '연락처' in i:
                        cont = getNext(i, ta)
                    if '사업장 소재지' in i:
                        loca = getNext(i, ta)
            except Exception as ex:
                error.append(ex)

            print('\t', c -1, name, cont, loca)

            try:
                if all([name, cont, loca]) is None:
                    if all([name, cont]):
                        final_info[0].append(name)
                        final_info[1].append(str(cont))
                        final_info[2].append('')
                        print('\t',name, cont)
                    else: 
    #                         print('\tnothing')
                        continue
                else:
                    final_info[0].append(name)
                    final_info[1].append(str(cont))
                    final_info[2].append(loca)
            except:
                print('except', name, cont, loca)
        browser.close()
        delCookie()
except Exception as ex:
    print(ex)


df = pd.DataFrame({'대표자': final_info[0],
                  '전화번호': final_info[1],
                  '영업소재지': final_info[2]
                  })

df = df.fillna('null')

df = df.drop_duplicates()

df.reset_index(inplace = True, drop = True)
df.to_csv(f'coupang_{keyword}.csv', encoding='utf-8-sig', index = False)
    

36 0 같이 https://www.coupang.com/vp/products/6628677182?itemId=15103371032&vendorItemId=82325336300&sourceType=srp_product_ads&clickEventId=bd391313-03a9-43b2-bba9-a88fc00a9634&korePlacement=15&koreSubPlacement=1&clickEventId=bd391313-03a9-43b2-bba9-a88fc00a9634&korePlacement=15&koreSubPlacement=1&q=%EA%B0%99%EC%9D%B4&itemsCount=36&searchId=908c976b626c4620977a0ae2d6c186ff&rank=0


----------------

In [32]:
df = pd.DataFrame({'대표자': final_info[0],
              '전화번호': final_info[1],
              '영업소재지': final_info[2]
              })

df = df.fillna('null')

df = df.drop_duplicates()

df.reset_index(inplace = True, drop = True)
df.to_csv(f'coupang_{keyword}.csv', encoding='utf-8-sig', index = False)
df.to_csv(f'coupang_error_{keyword}.csv', encoding='utf-8-sig', index = False)